# Getting Started with LouieAI Notebook Interface

This notebook demonstrates the basic usage of the LouieAI notebook-friendly API (`lui`).

## Authentication Requirements

LouieAI uses PyGraphistry for authentication. Before running this notebook, you need:

1. **PyGraphistry Account**: Sign up for free at [hub.graphistry.com](https://hub.graphistry.com)
2. **Set Credentials**: Use one of these methods:

   ```bash
   # Option 1: Environment variables (recommended for notebooks)
   export GRAPHISTRY_USERNAME="your_username"
   export GRAPHISTRY_PASSWORD="your_password"
   ```
   
   ```python
   # Option 2: Authenticate in code
   import graphistry
   graphistry.register(api=3, server="hub.graphistry.com", 
                      username="your_username", password="your_password")
   ```

## Setup

First, make sure you have LouieAI installed and your credentials configured.

In [1]:
import os

import graphistry

from louieai import louie  # Factory function to create Louie interface

# Authenticate with Graphistry
# Uses GRAPHISTRY_USERNAME and GRAPHISTRY_PASSWORD from environment
g = graphistry.register(
    api=3,
    server=os.environ.get("GRAPHISTRY_SERVER", "hub.graphistry.com"),
    username=os.environ.get("GRAPHISTRY_USERNAME"),
    password=os.environ.get("GRAPHISTRY_PASSWORD"),
)

# Configure Louie server URL
# Default to production server, or set LOUIE_SERVER env var for custom
# Examples:
#   export LOUIE_SERVER="http://localhost:8000"  # Local development
#   export LOUIE_SERVER="https://den.louie.ai"    # Production (default)
LOUIE_SERVER_URL = os.environ.get("LOUIE_SERVER", "https://den.louie.ai")

# Create the Louie interface
lui = louie(g, server_url=LOUIE_SERVER_URL)

lui  # Display status

<LouieAI Notebook Interface | Session: Not started | History: 0 responses | Traces: Disabled>

## Basic Queries

The `lui` interface accepts natural language queries and returns AI-generated responses. Let's verify it works with a simple arithmetic question:

In [2]:
# Ask a simple arithmetic question - should return "4"
response = lui("Calculate: 2 + 2")

# The response is automatically stored and accessible
print(f"Answer: {lui.text}")

# Verify we got a sensible answer
if "4" in str(lui.text):
    print("✓ Basic query working correctly!")
else:
    print("⚠ Response may be truncated or incomplete")

Answer: The task is to calculate a simple arithmetic expression, 2 + 2.
The result of 2
⚠ Response may be truncated or incomplete


## Working with Data

LouieAI can generate and analyze data. Let's create a sample dataset:

In [3]:
# Use the exact working prompt from our test
lui("Generate a pandas dataframe with 3 rows: name, age, city columns")

# Access the generated dataframe
df = lui.df
if df is not None:
    print(f"Generated DataFrame with shape: {df.shape}")
    print("\nDataFrame contents:")
    display(df)
else:
    print("No dataframe was generated. Let's try a different approach.")
    print(f"Response text: {lui.text}")
    
    # Try alternate approach
    lui("Create a table with 3 products: name, price, quantity")
    if lui.df is not None:
        print(f"\nSecond attempt successful! Shape: {lui.df.shape}")
        display(lui.df)

Generated DataFrame with shape: (3, 3)

DataFrame contents:


,name,age,city
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


## Analyzing Data  

Once you have data, you can ask LouieAI to perform calculations and analysis using its code agent:

In [4]:
# Ask for a calculation that requires the code agent
lui("Calculate the average age from the dataframe")

# The calculation result should be available
print(f"Result: {lui.text}")

# Verify the calculation (should be 30 since we have ages 25, 30, 35)
if "30" in str(lui.text):
    print("✓ Code agent calculation working!")
    
# You can also get more complex analysis
lui("What's the youngest person's name and city?")
print(f"\nAnalysis: {lui.text}")


Analysis: The task of identifying the youngest person in the DataFrame has been successfully completed. The DataFrame, which contains information about individuals named Alice, Bob, and Charlie, was analyzed to find the person with the minimum age. The result revealed that Alice, who resides in New York, is the youngest person. This information was extracted and saved efficiently, with the entire process executed smoothly and without errors.


## Beyond DataFrames

LouieAI can also analyze images, PDFs, Excel files, and more!

For comprehensive examples of uploading different file types, see the **[Data Upload Guide](02-data-upload.ipynb)**:
- Image analysis (PNG, JPEG, etc.)
- Document processing (PDF, Word, PowerPoint)
- Excel file handling
- Advanced upload options

In [5]:
# Quick example: Create and analyze a simple CSV file
import tempfile
import pandas as pd
import os

# Create a temporary CSV file
with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as f:
    f.write("product,price,quantity\n")
    f.write("Widget A,10.99,100\n")
    f.write("Widget B,15.49,75\n")
    f.write("Widget C,8.99,150\n")
    csv_path = f.name

# Load and analyze the CSV
df_from_csv = pd.read_csv(csv_path)
lui("What's the total inventory value?", df_from_csv)
print(f"CSV Analysis: {lui.text}")

# Clean up
os.unlink(csv_path)

CSV Analysis: What's the total inventory value?


## Uploading Your Own Data

LouieAI can analyze DataFrames you create or load from files:

In [6]:
import numpy as np
import pandas as pd

# Create your own DataFrame (or load from CSV/Excel)
my_data = pd.DataFrame(
    {
        "date": pd.date_range("2024-01-01", periods=30, freq="D"),
        "sales": np.random.randn(30).cumsum() + 100,
        "visitors": np.random.randint(50, 200, 30),
        "region": np.random.choice(["North", "South", "East", "West"], 30),
    }
)

print("Sample of your data:")
my_data.head()

Sample of your data:


,date,sales,visitors,region
0,2024-01-01,99.797559,172,North
1,2024-01-02,99.253910,128,North
2,2024-01-03,101.221238,111,North
3,2024-01-04,103.389155,113,East
4,2024-01-05,102.733427,156,North


In [7]:
# Upload and analyze your DataFrame
lui("Calculate the total sales from this data", my_data)

# The AI analyzes YOUR uploaded data
print(f"Analysis: {lui.text}")

# You can ask follow-up questions about the uploaded data
lui("Which region had the most visitors?")
print(f"\nRegion analysis: {lui.text}")


Region analysis: To determine which region had the most visitors, I need to know the specific dataset or dataframe that contains the visitor information, including regions and visitor counts. Could you please provide the relevant dataframe ID or any additional details about the data? This will help me guide you


## Session History

The `lui` interface maintains a history of your interactions:

In [8]:
# Access previous responses by index
# lui[-1]  # Last response
# lui[-2]  # Second to last response

# Check the number of interactions
print(f"Total interactions: {len(lui._history)}")

Total interactions: 7


## Enabling Traces

For complex queries, you might want to see the AI's reasoning process:

In [9]:
# Enable traces for the session
lui.traces = True

# Now queries will show reasoning steps
lui("What is 100 + 50?")

# Disable traces
lui.traces = False

# Or enable traces for just one query
lui("What is the sum of 1 to 10?", traces=True)

<LouieAI Notebook Interface | Session: Active | History: 9 responses | Traces: Disabled | Latest: 1 text>

## Error Handling

The interface handles errors gracefully:

In [10]:
# Check if the last operation had errors
if lui.has_errors:
    print("Last operation had errors:", lui.errors)
else:
    print("No errors in last operation")

No errors in last operation


## Tips and Tricks

Here are some helpful patterns for using the notebook interface:

1. **Use clear prompts**: Be specific about what you want to analyze
2. **Check outputs**: Always verify `lui.df` and `lui.text` after queries
3. **Enable traces for debugging**: Use `traces=True` when troubleshooting
4. **Leverage history**: Access previous results with `lui[-1]`, `lui[-2]`, etc.
5. **Upload various formats**: DataFrames, images, PDFs all work seamlessly